In [2]:
from transformers import VitsModel, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from IPython.display import Audio
import os
import time

/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(f"current working dir ->{os.getcwd()}")
#os.chdir('..')
print(f"current working dir ->{os.getcwd()}")

current working dir ->/workspace/Cohort8-Ransom-Kuti-Ladipo/yb2audio/notebooks
current working dir ->/workspace/Cohort8-Ransom-Kuti-Ladipo/yb2audio


In [22]:
mms_model_name = "facebook/mms-tts-yor"
mms_model = VitsModel.from_pretrained(mms_model_name)
mms_tokenizer = AutoTokenizer.from_pretrained(mms_model_name)
nllb_model_name = 'facebook/nllb-200-distilled-600M'
youruba_lang = 'yor_Latn'
nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name)

Some weights of the model checkpoint at facebook/mms-tts-yor were not used when initializing VitsModel: ['posterior_encoder.wavenet.in_layers.3.weight_v', 'posterior_encoder.wavenet.res_skip_layers.3.weight_g', 'flow.flows.3.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.res_skip_layers.3.weight_g', 'posterior_encoder.wavenet.in_layers.2.weight_v', 'flow.flows.2.wavenet.in_layers.1.weight_g', 'flow.flows.3.wavenet.res_skip_layers.2.weight_v', 'posterior_encoder.wavenet.in_layers.0.weight_v', 'posterior_encoder.wavenet.in_layers.12.weight_g', 'posterior_encoder.wavenet.res_skip_layers.4.weight_v', 'flow.flows.1.wavenet.res_skip_layers.1.weight_v', 'posterior_encoder.wavenet.res_skip_layers.7.weight_g', 'posterior_encoder.wavenet.in_layers.7.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.3.wavenet.in_layers.1.weight_g', 'flow.flows.1.wavenet.in_layers.2.weight_v', 'posterior_encoder.wavenet.in_layers.10.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_

In [5]:
def MMS_model_infer(text):
    inputs = mms_tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        output = mms_model(**inputs).waveform
    return Audio(output, rate=mms_model.config.sampling_rate)

def NLLB_infer(eng_text):
    inputs = nllb_tokenizer(eng_text, return_tensors='pt')
    translate_token = nllb_model.generate(**inputs, forced_bos_token_id = nllb_tokenizer.lang_code_to_id[youruba_lang],
                                          max_length = 50)
    outputs = nllb_tokenizer.batch_decode(translate_token, skip_special_tokens=True)[0]
    return outputs

def get_directory_size(directory_path):
    total_size = 0
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                total_size += entry.stat().st_size
            elif entry.is_dir():
                total_size += get_directory_size(entry.path)
    return round(total_size / (1024 * 1024), 2)

In [24]:
time_1 = time.time()
yor = NLLB_infer("i will go to work,on sunday,for 50,000 naira")
print(yor)
out = MMS_model_infer(yor)
time_2 = time.time()
print(time_2 - time_1)
out

Mo máa lọ síbi iṣẹ́ lọ́jọ́ Sunday, fún ẹ̀ẹ́dẹ́gbẹ̀rún márùn-ún naira
3.4393179416656494


Quantizattion of NLLB

In [ ]:
nllb_model_name = 'facebook/nllb-200-distilled-600M'
youruba_lang = 'yor_Latn'
nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model, load)